In [29]:
!python -V

Python 3.9.19


In [30]:
import pandas as pd

In [31]:
import pickle

In [32]:
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [34]:
import os

current_directory = os.getcwd()
print(current_directory)

c:\Users\DimiP\Documents\GitHub\0 REPOS My Learning\mlops-zoomcamp\02-experiment-tracking


In [35]:
import subprocess
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [37]:
import mlflow
from mlflow.tracking import MlflowClient

#mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
#client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


In [39]:
# marche pas :(


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment3")

2024/06/06 23:56:17 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment3' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///c:/Users/DimiP/Documents/GitHub/0 REPOS My '
 'Learning/mlops-zoomcamp/02-experiment-tracking/mlruns/6'), creation_time=1717710977771, experiment_id='6', last_update_time=1717710977771, lifecycle_stage='active', name='nyc-taxi-experiment3', tags={}>

In [40]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [41]:
from pathlib import Path

folder_path = Path("../01-intro/data")

parquet_files = [f for f in folder_path.iterdir() if f.is_file() and f.suffix == ".parquet"]

# open first file as df




In [42]:
df_train = read_dataframe(parquet_files[0])[0:1000]
df_val = read_dataframe(parquet_files[1])[0:1000]

In [43]:
len(df_train), len(df_val)

(1000, 1000)

In [44]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [45]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [46]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [47]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


6.375537699392879

In [48]:
# open lin_reg.bin in parent folder /01-intro/models

with open('../01-intro/models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [51]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Dimitri")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../01-intro/models/lin_reg.bin", artifact_path="models_pickle")

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [52]:
import xgboost as xgb

In [53]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [54]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [55]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [57]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

[0]	validation-rmse:7.64379                          
[1]	validation-rmse:6.56073                          
[2]	validation-rmse:5.98100                          
[3]	validation-rmse:5.77917                          
[4]	validation-rmse:5.67388                          
[5]	validation-rmse:5.69406                          
[6]	validation-rmse:5.74340                          
[7]	validation-rmse:5.78164                          
[8]	validation-rmse:5.85521                          
[9]	validation-rmse:5.87734                          
[10]	validation-rmse:5.91116                         
[11]	validation-rmse:5.93262                         
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\xgboost\core.py:160: UserWarning: [00:33:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[12]	validation-rmse:5.96315                         
[13]	validation-rmse:6.00220                         
[14]	validation-rmse:6.00194                         
[15]	validation-rmse:6.00030                         
[16]	validation-rmse:6.03430                         
[17]	validation-rmse:6.04510                         
[18]	validation-rmse:6.04939                         
[19]	validation-rmse:6.06491                         
[20]	validation-rmse:6.05901                         
[21]	validation-rmse:6.06358                         
[22]	validation-rmse:6.07085                         
[23]	validation-rmse:6.08500                         
[24]	validation-rmse:6.07116                         
[25]	validation-rmse:6.08886                         
[26]	validation-rmse:6.10516                         
[27]	validation-rmse:6.11010                         
[28]	validation-rmse:6.10363                         
[29]	validation-rmse:6.10993                         
[30]	validation-rmse:6.11880

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\xgboost\core.py:160: UserWarning: [00:33:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



...

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\xgboost\core.py:160: UserWarning: [00:33:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.70768                                                  
[1]	validation-rmse:8.03299                                                  
[2]	validation-rmse:7.49333                                                  
[3]	validation-rmse:7.03820                                                  
[4]	validation-rmse:6.68390                                                  
[5]	validation-rmse:6.35971                                                  
[6]	validation-rmse:6.10912                                                  
[7]	validation-rmse:5.86425                                                  
[8]	validation-rmse:5.65348                                                  
[9]	validation-rmse:5.51570                                                  
[10]	validation-rmse:5.42146                                                 
[11]	validation-rmse:5.34421                                                 
[12]	validation-rmse:5.28510                                    

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\xgboost\core.py:160: UserWarning: [00:33:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.65924                                                    
[1]	validation-rmse:8.02427                                                    
[2]	validation-rmse:7.49616                                                    
[3]	validation-rmse:7.02544                                                    
[4]	validation-rmse:6.61505                                                    
[5]	validation-rmse:6.29341                                                    
[6]	validation-rmse:6.03367                                                    
[7]	validation-rmse:5.79104                                                    
[8]	validation-rmse:5.60926                                                    
[9]	validation-rmse:5.48532                                                    
[10]	validation-rmse:5.38066                                                   
[11]	validation-rmse:5.30440                                                   
[12]	validation-rmse:5.23323            

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [58]:
mlflow.xgboost.autolog(disable=True)

In [61]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:8.72943
[1]	validation-rmse:8.15085
[2]	validation-rmse:7.66420
[3]	validation-rmse:7.26413
[4]	validation-rmse:6.94302
[5]	validation-rmse:6.69356
[6]	validation-rmse:6.49649
[7]	validation-rmse:6.34504


c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\xgboost\core.py:160: UserWarning: [00:56:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[8]	validation-rmse:6.23496
[9]	validation-rmse:6.16636
[10]	validation-rmse:6.12810
[11]	validation-rmse:6.11084
[12]	validation-rmse:6.10613
[13]	validation-rmse:6.11979
[14]	validation-rmse:6.13913
[15]	validation-rmse:6.18047
[16]	validation-rmse:6.21927
[17]	validation-rmse:6.24609
[18]	validation-rmse:6.27578
[19]	validation-rmse:6.31496
[20]	validation-rmse:6.35809
[21]	validation-rmse:6.39829
[22]	validation-rmse:6.42866
[23]	validation-rmse:6.45779
[24]	validation-rmse:6.49667
[25]	validation-rmse:6.52002
[26]	validation-rmse:6.55800
[27]	validation-rmse:6.58013
[28]	validation-rmse:6.60116
[29]	validation-rmse:6.63457
[30]	validation-rmse:6.65322
[31]	validation-rmse:6.67188
[32]	validation-rmse:6.68594
[33]	validation-rmse:6.70283
[34]	validation-rmse:6.71955
[35]	validation-rmse:6.73514
[36]	validation-rmse:6.75069
[37]	validation-rmse:6.77267
[38]	validation-rmse:6.79163
[39]	validation-rmse:6.82132
[40]	validation-rmse:6.83214
[41]	validation-rmse:6.84788
[42]	validation-

c:\Users\DimiP\miniconda3\envs\exp-tracking_env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
